In [1]:
# !pip install category_encoders

In [2]:
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn import set_config
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

In [3]:
df = pd.read_csv("HW.csv")

In [4]:
df

,Tipster,Date,Track,Horse,Bet Type,Odds,Result,TipsterActive
0,Tipster A,24-07-2015,Ascot,Fredricka,Win,8.00,Lose,True
1,Tipster A,24-07-2015,Thirsk,Spend A Penny,Win,4.50,Lose,True
2,Tipster A,24-07-2015,York,Straightothepoint,Win,7.00,Lose,True
3,Tipster A,24-07-2015,Newmarket,Miss Inga Sock,Win,5.00,Lose,True
4,Tipster A,25-07-2015,Ascot,Peril,Win,4.33,Win,True
...,...,...,...,...,...,...,...,...
38243,Tipster E1,02-04-2016,Kempton,Solar Flair,Win,7.00,Lose,False
38244,Tipster E1,02-04-2016,Doncaster,Express Himself,Each Way,12.00,Lose,False
38245,Tipster E1,02-04-2016,Doncaster,Jack Dexter,Win,7.00,Lose,False
38246,Tipster E1,02-04-2016,Kelso,Just Cameron,Win,4.33,Lose,False


In [5]:
df.shape

(38248, 8)

In [6]:
## Checking for the missing values
df.isna().sum()

Tipster          0
Date             0
Track            0
Horse            0
Bet Type         0
Odds             0
Result           0
TipsterActive    0
dtype: int64

In [7]:
df["Horse"].unique()

array(['Fredricka', 'Spend A Penny', 'Straightothepoint', ...,
       'Connetable', 'Allchilledout', 'Dream Farr'], dtype=object)

In [8]:
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')

In [9]:
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day

In [10]:
df.head()

,Tipster,Date,Track,Horse,Bet Type,Odds,Result,TipsterActive,Year,Month,Day
0,Tipster A,2015-07-24,Ascot,Fredricka,Win,8.00,Lose,True,2015,7,24
1,Tipster A,2015-07-24,Thirsk,Spend A Penny,Win,4.50,Lose,True,2015,7,24
2,Tipster A,2015-07-24,York,Straightothepoint,Win,7.00,Lose,True,2015,7,24
3,Tipster A,2015-07-24,Newmarket,Miss Inga Sock,Win,5.00,Lose,True,2015,7,24
4,Tipster A,2015-07-25,Ascot,Peril,Win,4.33,Win,True,2015,7,25


In [11]:
## Dropping the date column
df = df.drop("Date", axis = 1)

In [12]:
df

,Tipster,Track,Horse,Bet Type,Odds,Result,TipsterActive,Year,Month,Day
0,Tipster A,Ascot,Fredricka,Win,8.00,Lose,True,2015,7,24
1,Tipster A,Thirsk,Spend A Penny,Win,4.50,Lose,True,2015,7,24
2,Tipster A,York,Straightothepoint,Win,7.00,Lose,True,2015,7,24
3,Tipster A,Newmarket,Miss Inga Sock,Win,5.00,Lose,True,2015,7,24
4,Tipster A,Ascot,Peril,Win,4.33,Win,True,2015,7,25
...,...,...,...,...,...,...,...,...,...,...
38243,Tipster E1,Kempton,Solar Flair,Win,7.00,Lose,False,2016,4,2
38244,Tipster E1,Doncaster,Express Himself,Each Way,12.00,Lose,False,2016,4,2
38245,Tipster E1,Doncaster,Jack Dexter,Win,7.00,Lose,False,2016,4,2
38246,Tipster E1,Kelso,Just Cameron,Win,4.33,Lose,False,2016,4,2


In [13]:
## Spliting data
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['Result']),df['Result'],test_size=0.2,random_state=42)

In [14]:
X_train

,Tipster,Track,Horse,Bet Type,Odds,TipsterActive,Year,Month,Day
4812,Tipster E,Yarmouth,Reaffirmed,Win,5.0,True,2014,6,26
34052,Tipster B1,Ascot,Polarisation,Win,9.0,False,2015,9,5
23400,Tipster X,Taunton,Elsafeer,Win,7.0,True,2013,11,28
21464,Tipster T,Newmarket,Riflescope,Win,26.0,True,2016,7,7
18200,Tipster R,Uttoxeter,Catching On,Win,5.0,True,2015,3,14
...,...,...,...,...,...,...,...,...,...
6265,Tipster E,Chepstow,Lac Sacre,Win,5.5,True,2015,5,19
11284,Tipster J,Shatin,Nuovo Record,Win,13.0,True,2016,4,24
38158,Tipster E1,SouthWell,Alco Sivola,Win,6.0,False,2015,10,22
860,Tipster A,Newmarket,Dougan,Win,7.5,True,2016,5,1


In [15]:
## Checking for the unique values
df["Bet Type"].unique()

array(['Win', 'Each Way', 'win'], dtype=object)

In [16]:
## Replacing "win" to "Win"
df["Bet Type"] = df["Bet Type"].replace("win", "Win")

In [17]:
## Checking the result, if replaced or not
df["Bet Type"].unique()

array(['Win', 'Each Way'], dtype=object)

In [18]:
## Unique values for all the columns

In [19]:
df["Tipster"].unique()

array(['Tipster A', 'Tipster B', 'Tipster C', 'Tipster D', 'Tipster E',
       'Tipster F', 'Tipster G', 'Tipster H', 'Tipster I', 'Tipster J',
       'Tipster K', 'Tipster L', 'Tipster M', 'Tipster N', 'Tipster O',
       'Tipster P', 'Tipster Q', 'Tipster R', 'Tipster S', 'Tipster T',
       'Tipster U', 'Tipster V', 'Tipster W', 'Tipster X', 'Tipster Y',
       'Tipster Z', 'Tipster A1', 'Tipster B1', 'Tipster C1',
       'Tipster D1', 'Tipster E1'], dtype=object)

In [20]:
df["Track"].unique()

array(['Ascot', 'Thirsk', 'York', 'Newmarket', 'Newcastle', 'Lingfield',
       'Carlisle', 'Pontefract', 'Uttoxeter', 'Hamilton', 'Chepstow',
       'Salisbury', 'Catterick', 'Chelmsford', 'Brighton',
       'Wolverhampton', 'Newbury', 'Market Rasen', 'Tramore', 'Leicester',
       'Ffos Las', 'Worcester', 'Sandown', 'Deauville', 'Curragh',
       'Stratford', 'Goodwood', 'Beverley', 'Doncaster', 'Chester',
       'Leopardstown', 'Bath', 'Yarmouth', 'Ayr', 'Kempton', 'Haydock',
       'Nottingham', 'Dundalk', 'Sedgefield', 'Huntingdon', 'Wetherby',
       'Wincanton', 'Fakenham', 'Cork', 'Naas', 'Windsor', 'Exeter',
       'Galway', 'Navan', 'Ludlow', 'Cheltenham', 'Aintree', 'Listowel',
       'Limerick', 'Plumpton', 'Warwick', 'Thurles', 'Towcester',
       'Musselburgh', 'Hexham', 'Taunton', 'Punchestown', 'Fontwell',
       'SouthWell', 'Fairyhouse', 'Bangor', 'Tipperary', 'Longchamp',
       'Meydan', 'Gowran Park', 'Cartmel', 'Down Royal', 'Kelso',
       'Downpatrick', 'Ripon',

In [21]:
df["TipsterActive"].unique()

array([ True, False])

## Trial

In [22]:
df

,Tipster,Track,Horse,Bet Type,Odds,Result,TipsterActive,Year,Month,Day
0,Tipster A,Ascot,Fredricka,Win,8.00,Lose,True,2015,7,24
1,Tipster A,Thirsk,Spend A Penny,Win,4.50,Lose,True,2015,7,24
2,Tipster A,York,Straightothepoint,Win,7.00,Lose,True,2015,7,24
3,Tipster A,Newmarket,Miss Inga Sock,Win,5.00,Lose,True,2015,7,24
4,Tipster A,Ascot,Peril,Win,4.33,Win,True,2015,7,25
...,...,...,...,...,...,...,...,...,...,...
38243,Tipster E1,Kempton,Solar Flair,Win,7.00,Lose,False,2016,4,2
38244,Tipster E1,Doncaster,Express Himself,Each Way,12.00,Lose,False,2016,4,2
38245,Tipster E1,Doncaster,Jack Dexter,Win,7.00,Lose,False,2016,4,2
38246,Tipster E1,Kelso,Just Cameron,Win,4.33,Lose,False,2016,4,2


##### enc_Horse = (df.groupby('Track').size())/len(df)
##### enc_Horse
##### df['Track_encode']= df['Track'].apply(lambda x : enc_Horse[x])


##### enc_Horse = (df.groupby('Tipster').size())/len(df)
##### enc_Horse
##### df['Tipster_encode']= df['Tipster'].apply(lambda x : enc_Horse[x])


In [23]:
## Pipeline Preparation..

In [24]:
## Defining a function to perform frequency Encoding and then using it in Column transformer

def encode_column(column):
    series_column = column.squeeze()  
    enc_column = series_column.map(series_column.value_counts() / len(series_column))
    return enc_column.to_frame()  

In [25]:
## Using Column Transformer for encoding on the the columns.

In [26]:
trf1 = ColumnTransformer(
    transformers=[
        ('Horse_encode', FunctionTransformer(encode_column, validate=False), ['Horse']),
        ('Track_encode', FunctionTransformer(encode_column, validate=False), ['Track']),
        ('Tipster_encode', FunctionTransformer(encode_column, validate=False), ['Tipster']),
        ('ohe_TipsterActive', OneHotEncoder(handle_unknown='ignore'), [3, 6]),
        
    ],
    remainder='passthrough')

In [27]:
label_encoder = preprocessing.LabelEncoder()
y_trans = label_encoder.fit_transform(y_train)

In [28]:
df

,Tipster,Track,Horse,Bet Type,Odds,Result,TipsterActive,Year,Month,Day
0,Tipster A,Ascot,Fredricka,Win,8.00,Lose,True,2015,7,24
1,Tipster A,Thirsk,Spend A Penny,Win,4.50,Lose,True,2015,7,24
2,Tipster A,York,Straightothepoint,Win,7.00,Lose,True,2015,7,24
3,Tipster A,Newmarket,Miss Inga Sock,Win,5.00,Lose,True,2015,7,24
4,Tipster A,Ascot,Peril,Win,4.33,Win,True,2015,7,25
...,...,...,...,...,...,...,...,...,...,...
38243,Tipster E1,Kempton,Solar Flair,Win,7.00,Lose,False,2016,4,2
38244,Tipster E1,Doncaster,Express Himself,Each Way,12.00,Lose,False,2016,4,2
38245,Tipster E1,Doncaster,Jack Dexter,Win,7.00,Lose,False,2016,4,2
38246,Tipster E1,Kelso,Just Cameron,Win,4.33,Lose,False,2016,4,2


In [29]:
trf2 = ColumnTransformer([
    ('scale',StandardScaler(),slice(0,11))
])

In [30]:
clf = LogisticRegression()

In [31]:
## Pipeline

In [32]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('clf',clf)
])

In [33]:
# train
pipe.fit(X_train,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('Horse_encode',
                                                  FunctionTransformer(func=<function encode_column at 0x7fc9802f4ee0>),
                                                  ['Horse']),
                                                 ('Track_encode',
                                                  FunctionTransformer(func=<function encode_column at 0x7fc9802f4ee0>),
                                                  ['Track']),
                                                 ('Tipster_encode',
                                                  FunctionTransformer(func=<function encode_column at 0x7fc9802f4ee0>),
                                                  ['Tipster']),
                                                 ('ohe_TipsterActive',
                                                  OneHotEncoder(handle_unknown='ignore'),
      

In [34]:
from sklearn import set_config
set_config(display='diagram')

In [35]:
# Predict, testing Accuracy
y_pred = pipe.predict(X_test)

In [36]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8006535947712419

In [37]:
# Predict for training accuracy
y_pred1 = pipe.predict(X_train)

In [38]:
from sklearn.metrics import accuracy_score
accuracy_score(y_train,y_pred1)

0.79848356101706

## Cross Validation

In [39]:
# cross validation using cross_val_score
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy')

array([0.79869281, 0.79787582, 0.79624183, 0.79702566, 0.80029417])

In [40]:
# cross validation using cross_val_score
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy').mean()

0.7980260562033824

## Grid Search CV

In [41]:
# gridsearchcv
param_grid = {
    "clf__penalty": ['l1', 'l2'],
    "clf__C": [0.001, 0.01, 0.1, 1, 10, 100]
}

In [42]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy')
grid.fit(X_train, y_train)

/Users/sid24082/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/sid24082/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/sid24082/opt/anaconda3/lib/python3.9/site-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/sid24082/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 1461, i

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('Horse_encode',
                                                                         FunctionTransformer(func=<function encode_column at 0x7fc9802f4ee0>),
                                                                         ['Horse']),
                                                                        ('Track_encode',
                                                                         FunctionTransformer(func=<function encode_column at 0x7fc9802f4ee0>),
                                                                         ['Track']),
                                                                        ('Tipster_encode',
                                                                         FunctionTransformer(func=<function encode_column at 0x7fc9802f4ee0>),
                                                                         ['Tipster']),
                                                                        ('ohe_TipsterActive',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         [3,
                                                                          6])])),
                                       ('trf2',
                                        ColumnTransformer(transformers=[('scale',
                                                                         StandardScaler(),
                                                                         slice(0, 11, None))])),
                                       ('clf', LogisticRegression())]),
             param_grid={'clf__C': [0.001, 0.01, 0.1, 1, 10, 100],
                         'clf__penalty': ['l1', 'l2']},
             scoring='accuracy')

In [43]:
grid.best_score_

0.798646971236062

In [44]:
grid.best_params_

{'clf__C': 0.001, 'clf__penalty': 'l2'}

## Pickel File

In [45]:
# export 
import pickle
pickle.dump(pipe,open('logis_reg.pkl','wb'))

In [46]:
lr_pipe = pickle.load(open('logis_reg.pkl','rb'))

In [47]:
X_train

,Tipster,Track,Horse,Bet Type,Odds,TipsterActive,Year,Month,Day
4812,Tipster E,Yarmouth,Reaffirmed,Win,5.0,True,2014,6,26
34052,Tipster B1,Ascot,Polarisation,Win,9.0,False,2015,9,5
23400,Tipster X,Taunton,Elsafeer,Win,7.0,True,2013,11,28
21464,Tipster T,Newmarket,Riflescope,Win,26.0,True,2016,7,7
18200,Tipster R,Uttoxeter,Catching On,Win,5.0,True,2015,3,14
...,...,...,...,...,...,...,...,...,...
6265,Tipster E,Chepstow,Lac Sacre,Win,5.5,True,2015,5,19
11284,Tipster J,Shatin,Nuovo Record,Win,13.0,True,2016,4,24
38158,Tipster E1,SouthWell,Alco Sivola,Win,6.0,False,2015,10,22
860,Tipster A,Newmarket,Dougan,Win,7.5,True,2016,5,1


In [48]:
# Assume user input
test_data = np.array([['Tipster E1', 'Thirsk', 'Solar Flair', 'Win', 8.0, True, 2016, 4, 2]],dtype=object).reshape(1,9)

In [49]:
# test_df = pd.DataFrame(test_data, columns=X_train.columns)

In [50]:
lr_pipe.predict(test_data)

ValueError: Specifying the columns using strings is only supported for pandas DataFrames